# Exploring the LSTM model for stock price prediction

### Load the data


In [1]:
import pandas as pd
import datetime
from tensorflow import keras
import data_engineering


In [2]:
keras.__version__

'2.6.0'

In [3]:
import numpy
numpy.__version__

'1.26.4'

In [3]:
# load data
df_dict = data_engineering.separate_by_stock()

train, test = data_engineering.train_test_split(df_dict['AAPL'])


In [4]:
train.describe()

,sentiment_tot,finvader,Open,Close
count,991.000000,991.000000,1007.000000,1007.000000
mean,0.153922,0.127202,112.703139,112.807771
std,0.226626,0.198091,41.186390,41.188441
min,-0.824500,-0.841400,40.936194,41.460152
25%,0.018710,0.009069,70.145290,70.436131
50%,0.158011,0.123109,124.980357,124.767517
75%,0.290760,0.245067,146.183655,146.419571
max,0.932500,0.904200,180.092530,179.481140


In [5]:
train_new = train.iloc[:,2:4]
train_new = train_new.fillna(-5)
train_new.head()

,finvader,Open
Date,,
2019-03-01,-5.0,41.887973
2019-03-04,-5.0,42.226871
2019-03-05,-5.0,42.286956
2019-03-06,-5.0,41.981718
2019-03-07,-5.0,41.789428


In [6]:
# scale opening prices to be within -1,1
from sklearn.preprocessing import MinMaxScaler
train_new["Diff"] = train_new.Open.diff()
train_new['y'] = train_new["Diff"].shift(periods=-1)
train_new.Diff.iloc[0] = 0

In [7]:
train_new = train_new.drop(columns=['Open'])
train_new.head()

,finvader,Diff,y
Date,,,
2019-03-01,-5.0,0.000000,0.338898
2019-03-04,-5.0,0.338898,0.060085
2019-03-05,-5.0,0.060085,-0.305237
2019-03-06,-5.0,-0.305237,-0.192290
2019-03-07,-5.0,-0.192290,-0.853234


In [8]:
X = train_new.values
scaler = MinMaxScaler((-1,1))
X_scaled = X.copy()
X_scaled[:,1:] = scaler.fit_transform(X[:,1:])

In [9]:
X_scaled

array([[-5.        ,  0.0701893 ,  0.09853251],
       [-5.        ,  0.09853251,  0.0752144 ],
       [-5.        ,  0.0752144 ,  0.04466123],
       ...,
       [ 0.24793333, -0.17737843,  0.12003695],
       [ 0.0538625 ,  0.12003695,  0.01535645],
       [-0.28668571,  0.01535645,         nan]])

In [11]:
new_X = X_scaled.reshape(X.shape[0], 1, X.shape[1])
new_X

array([[[-5.        ,  0.0701893 ,  0.09853251]],

       [[-5.        ,  0.09853251,  0.0752144 ]],

       [[-5.        ,  0.0752144 ,  0.04466123]],

       ...,

       [[ 0.24793333, -0.17737843,  0.12003695]],

       [[ 0.0538625 ,  0.12003695,  0.01535645]],

       [[-0.28668571,  0.01535645,         nan]]])

## Creating the LSTM model

In [16]:
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [26]:
model = fit_lstm(X_scaled, 1, 5, 3)

In [35]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(2, 1, -1)
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [44]:
X_scaled.shape

(1007, 3)

In [37]:
forecast_lstm(model, 1, X_scaled[1][:-1].reshape((2,1,-1)))
model.predict()

ValueError: in user code:

    c:\Users\timal\anaconda3\envs\erdos_ds_2024_newsworthy\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    c:\Users\timal\anaconda3\envs\erdos_ds_2024_newsworthy\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\timal\anaconda3\envs\erdos_ds_2024_newsworthy\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\timal\anaconda3\envs\erdos_ds_2024_newsworthy\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\timal\anaconda3\envs\erdos_ds_2024_newsworthy\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, 

In [ ]:

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy
 
# date-time parsing function for loading the dataset
def parser(x):
 return datetime.strptime('190'+x, '%Y-%m')
 
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
 df = DataFrame(data)
 columns = [df.shift(i) for i in range(1, lag+1)]
 columns.append(df)
 df = concat(columns, axis=1)
 df.fillna(0, inplace=True)
 return df
 
# create a differenced series
def difference(dataset, interval=1):
 diff = list()
 for i in range(interval, len(dataset)):
 value = dataset[i] - dataset[i - interval]
 diff.append(value)
 return Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
 return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
 # fit scaler
 scaler = MinMaxScaler(feature_range=(-1, 1))
 scaler = scaler.fit(train)
 # transform train
 train = train.reshape(train.shape[0], train.shape[1])
 train_scaled = scaler.transform(train)
 # transform test
 test = test.reshape(test.shape[0], test.shape[1])
 test_scaled = scaler.transform(test)
 return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
 new_row = [x for x in X] + [value]
 array = numpy.array(new_row)
 array = array.reshape(1, len(array))
 inverted = scaler.inverse_transform(array)
 return inverted[0, -1]
 
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
 X, y = train[:, 0:-1], train[:, -1]
 X = X.reshape(X.shape[0], 1, X.shape[1])
 model = Sequential()
 model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
 model.add(Dense(1))
 model.compile(loss='mean_squared_error', optimizer='adam')
 for i in range(nb_epoch):
 model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
 model.reset_states()
 return model
 
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
 X = X.reshape(1, 1, len(X))
 yhat = model.predict(X, batch_size=batch_size)
 return yhat[0,0]
 
# load dataset
series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
 
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)
 
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values
 
# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]
 
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)
 
# fit the model
lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)
 
# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
 # make one-step forecast
 X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
 yhat = forecast_lstm(lstm_model, 1, X)
 # invert scaling
 yhat = invert_scale(scaler, X, yhat)
 # invert differencing
 yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
 # store forecast
 predictions.append(yhat)
 expected = raw_values[len(train) + i + 1]
 print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))
 
# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(raw_values[-12:])
pyplot.plot(predictions)
pyplot.show()